# Transform Vaisala RS41 radiosonde at CSU from L0 data (edt) to L1 csv format file (batch mode).

**The CSU Vaisala MW41 system for RS41 radiosonde exports the edt-format file in its own specific format.**

Hungjui Yu\
20210913

In [26]:
import glob
import time
import pandas as pd
import datetime as dt
import pytz
from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units

**Set RS L0 (edt) files path:**

In [27]:
RS_L0_file_path = '/Users/yuhungjui/Research/CSU_2021/PRECIP_2021/VaisalaRS41/Data/L0'


**Set output path:**

In [28]:
output_path = '/Users/yuhungjui/Research/CSU_2021/PRECIP_2021/VaisalaRS41/Data/L1_csv'


**Find all edt files in the directory:**

In [29]:
# for file_path_name in glob.glob(RS_L0_file_path + '/*/edt_*'):
#     print(file_path_name)

# os.listdir(RS_L0_file_path)
# os.walk(RS_L0_file_path)

file_path_name = '/Users/yuhungjui/Research/CSU_2021/PRECIP_2021/VaisalaRS41/Data/L0/20210626/edt_20210626_1741.txt'


**Load the edt file:**

In [33]:
def load_L0_edt(file_path_name):

    ## For sounding data:
    RS_df = pd.read_csv(file_path_name, sep='\s+', header=28, 
                        names=['Time','MSL','P','TC','TD','RH','WS','WD','AscRate','Lat','Lon','GPS_MSL'])
    # print(RS_df)

    ## For souding launch time:
    tz_utc = pytz.timezone('UTC')
    tz_fc = pytz.timezone('US/Mountain')
    RS_time = pd.read_csv(file_path_name, header=None, nrows=6)
    # print(RS_time.iloc[5][0][:])
    RS_launch_time = dt.datetime.strptime(RS_time.iloc[5][0][:],
                                          'Balloon release date and time                	%Y-%m-%dT%H:%M:%S')
    # print(launch_time)
    
    ## For sounding info:
    # RS_type = pd.read_csv(file_path_name, header=None, nrows=3)
    RS_sn = pd.read_csv(file_path_name, header=None, nrows=4).iloc[3][0][-8:]
    
    return RS_launch_time, RS_df, RS_sn

    # open_csv = open(file_path_name, 'r')
    # reader = csv.reader(open_csv)
    # for row in reader:
    #     print(row)

# RS_data = load_L0_edt(file_path_name)
# print(RS_data[2])

R1540499


**Convert from L0 data to L1 variables if needed:**

In [8]:
# def conversion_L0_L1(loaded_RS_file, RS_launch_time):

#     # launch_time = loaded_ST_file[0]
#     # L0_raw_data = loaded_ST_file[1]
    
#     tz_utc = pytz.timezone('UTC')

#     # Convert the data time to datetime object:
#     L0_raw_data['Time'] = pd.to_datetime(L0_raw_data['Time'], utc=tz_utc)

#     # Calculate dew-point temperature in raw data:
#     L0_raw_data['dT(degC)'] = dewpoint_from_relative_humidity((L0_raw_data['Temperature(degree C)'].to_numpy() * units.degC).to(units.K), L0_raw_data['Humidity(%)'].to_numpy() / 100.)

#     # Convert wind speed in raw data:
#     # L0_raw_data['WS(kts)'] = (L0_raw_data['Speed(km/hr)'].to_numpy() * units.kilometer_per_hour).to(units.knot)
#     L0_raw_data['WS(m/s)'] = (L0_raw_data['Speed(km/hr)'].to_numpy() * units.kilometer_per_hour).to(units.meter_per_second)

#     # Convert wind direction in raw data:
#     L0_raw_data.loc[L0_raw_data['Direction(degree)'] <= 180, 'WDIR'] = L0_raw_data['Direction(degree)'] + 180
#     L0_raw_data.loc[L0_raw_data['Direction(degree)'] > 180, 'WDIR'] = L0_raw_data['Direction(degree)'] - 180
#     L0_raw_data.loc[L0_raw_data['Speed(km/hr)'] == 0, 'WDIR'] = 0
    
#     # Find the index of launch time and convert L0 to L1 data:
#     L1_data = L0_raw_data[L0_raw_data['Time'] >= launch_time_utc].copy()
    
#     # Set Time(sec) in L1 data:
#     L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).dt.total_seconds()
#     # L1_data['Time(sec)'] = (L1_data['Time']-launch_time_utc).astype('timedelta64[s]')
    
#     return L1_data


In [107]:
def output_L1(loaded_RS_file, L1_csv_filename):
    
    # launch_time = loaded_RS_file[0]
    # L0_raw_data = loaded_RS_file[1]

    # Output L1 data (csv format):

    with open(L1_csv_filename, 'w') as file:

        # Required fields:
        file.write('FileFormat,CSV\n')
        file.write('Year,{}\n'.format(loaded_RS_file[0].year))
        file.write('Month,{:02d}\n'.format(loaded_RS_file[0].month))
        file.write('Day,{:02d}\n'.format(loaded_RS_file[0].day))
        file.write('Hour,{:02d}\n'.format(loaded_RS_file[0].hour))
        file.write('Minute,{:02d}\n'.format(loaded_RS_file[0].minute))
        file.write('Second,{:02d}\n'.format(loaded_RS_file[0].second))

        file.write('Ascending,"true"\n')

        # Optional fields:
        file.write('latitude,40.590000,"units=deg"\n')
        file.write('longitude,-105.141500,"units=deg"\n')
        file.write('altitude,1571.9,"units=m"\n')
        file.write('gpsaltitude,1571.9,"units=m"\n')
        file.write('project,"PRE-CIP-2021"\n')
        file.write('agency,"CSU"\n')
        file.write('sondeid,"{}"\n'.format(RS_no))
        file.write('sondetype,"Vaisala RS41-SGP"\n')
        file.write('launchsite,"Christman Field"\n')

        # Data headers:
        file.write('Fields,Time,Pressure,Temperature,RH,Speed,Direction,Latitude,Longitude,Altitude,gpsalt\n')
        file.write('Units,sec,mb,deg C,%,m/s,deg,deg,deg,m,m\n')

        # Data fields:

        for index, row in L1_data.iterrows():

            file.write('Data,%6.1f,%7.2f,%5.2f,%5.2f,%6.2f,%6.2f,%9.5f,%9.5f,,%7.1f\n'\
                       % (row['Time(sec)']\
                        , row['Pressure(hPa)']\
                        , row['Temperature(degree C)']\
                        , row['Humidity(%)']\
                        , row['WS(m/s)']\
                        , row['Direction(degree)']\
                        , row['Lat']\
                        , row['Lon']\
                        , row['Height(m)']\
                         )\
                      )

In [108]:
# %%
# MAIN PROGRAM:
t_start = time.process_time()

RS_info = read_in_RS_info(RS_info_file)

for index, row in RS_info.iterrows():

    if dt.datetime.strptime(str(row['Date']), '%Y%m%d') <= final_date:
        
        log_launchT = str(row['Date']) + str(row['Launch_T'])[:6]
        
        st_file_date_path = RS_L0_file_path + '/{}'.format(str(row['Date']))

        loaded_RS_file = load_st_file(str(row['RS_No'])[:4], log_launchT, st_file_date_path)
        
        L1_data = conversion_L0_L1(loaded_RS_file)
        
        L1_output_name = output_path + '/precip21_{}.RS_{}.csv'.format(str(row['Date']) + str(row['Nominal_T']), str(row['RS_No'])[:4])
        
        output_L1(str(row['RS_No'])[:4], loaded_RS_file, L1_data, L1_output_name)
        
        print(L1_output_name)

t_finish = time.process_time()
print('Process Time: %f Sec' % (t_finish - t_start))

/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021050318.ST_2968.csv
/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021050418.ST_2134.csv
/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021050518.ST_2133.csv


/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/numpy_func.py:303: RuntimeWarning: divide by zero encountered in log
  result_magnitude = func(*stripped_args, **stripped_kwargs)
/Users/yuhungjui/anaconda3/lib/python3.8/site-packages/pint/quantity.py:1237: RuntimeWarning: invalid value encountered in true_divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


/Users/yuhungjui/GoogleDrive_CSU/Research/CSU_2021/PRECIP_2021/StormTracker/Data/L1_csv/precip21_2021050818.ST_1841.csv
Process Time: 1.257042 Sec


In [25]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
